# Building a Basic Agent with LangGraph and Groq

In this notebook, we will build a simple agent using **LangGraph** and **Groq**'s Llama 3 model. The agent will be capable of performing basic arithmetic operations by deciding when to call specific tools.

## 1. Setup and Imports

First, we import the necessary libraries and load our environment variables (like the Groq API key).

In [ ]:
import operator
from typing import Literal, Annotated
from typing_extensions import TypedDict
from dotenv import load_dotenv

from langchain.chat_models import init_chat_model
from langchain.messages import SystemMessage, ToolMessage, HumanMessage, AnyMessage
from langchain.tools import tool
from langgraph.graph import StateGraph, START, END

# Load environment variables from .env file
load_dotenv()

## 2. Define Tools

We define the arithmetic tools (`multiply`, `add`, `divide`) that our agent can use. These are decorated with `@tool` to make them compatible with LangChain's tool binding system.

In [ ]:
# Define tools
@tool
def multiply(a: int, b: int) -> int:
    """Multiply `a` and `b`.

    Args:
        a: First int
        b: Second int
    """
    return a * b


@tool
def add(a: int, b: int) -> int:
    """Adds `a` and `b`.

    Args:
        a: First int
        b: Second int
    """
    return a + b


@tool
def divide(a: int, b: int) -> float:
    """Divide `a` and `b`.

    Args:
        a: First int
        b: Second int
    """
    return a / b


# Create a list of tools for the agent to bind to
tools = [add, multiply, divide]

# Create a mapping for the tool node to find the tool by name
tools_by_name = {tool.name: tool for tool in tools}

## 3. Define Model (Groq)

We need to initialize the chat model. Here we are using `llama-3.3-70b-versatile` hosted on Groq. We also bind the tools we defined earlier to the model, giving it the ability to call them.

In [ ]:
# We use init_chat_model with the Groq provider.
model = init_chat_model(
    "llama-3.3-70b-versatile",
    model_provider="groq",
    temperature=0
)

# Augment the LLM with tools
model_with_tools = model.bind_tools(tools)

## 4. Define State

We define the `MessagesState` which keeps track of the conversation history (list of messages) and any other state variables, like the number of LLM calls.

In [ ]:
class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    llm_calls: int

## 5. Define Nodes

Our graph consists of two main nodes:
1.  **`llm_call`**: This node invokes the LLM with the current state. The LLM decides whether to respond directly or request a tool call.
2.  **`tool_node`**: If the LLM requests a tool call, this node executes the tool and returns the result.

In [ ]:
def llm_call(state: MessagesState):
    """LLM decides whether to call a tool or not"""
    return {
        "messages": [
            model_with_tools.invoke(
                [
                    SystemMessage(
                        content="You are a helpful assistant tasked with performing arithmetic on a set of inputs."
                    )
                ]
                + state["messages"]
            )
        ],
        "llm_calls": state.get('llm_calls', 0) + 1
    }


def tool_node(state: MessagesState):
    """Performs the tool call"""
    result = []
    last_message = state["messages"][-1]
    
    for tool_call in last_message.tool_calls:
        tool = tools_by_name[tool_call["name"]]
        # Invoke the tool
        observation = tool.invoke(tool_call["args"])
        # Create a ToolMessage with the result
        result.append(ToolMessage(content=str(observation), tool_call_id=tool_call["id"]))
        
    return {"messages": result}

## 6. Define Logic (Conditional Edges)

We need a function to decide the flow of control. After the LLM runs, we check:
- If it generated a tool call -> Go to `tool_node`.
- If it just generated text -> End the execution.

In [ ]:
def should_continue(state: MessagesState) -> Literal["tool_node", END]:
    """Decide if we should continue the loop or stop based upon whether the LLM made a tool call"""
    messages = state["messages"]
    last_message = messages[-1]

    # If the LLM makes a tool call, then perform an action
    if last_message.tool_calls:
        return "tool_node"

    # Otherwise, we stop (reply to the user)
    return END

## 7. Build Agent Graph

Now we assemble the StateGraph. We add our nodes and define the edges to create the cycle: `llm_call` -> (conditional) -> `tool_node` -> `llm_call`.

In [ ]:
agent_builder = StateGraph(MessagesState)

# Add nodes
agent_builder.add_node("llm_call", llm_call)
agent_builder.add_node("tool_node", tool_node)

# Add edges
agent_builder.add_edge(START, "llm_call")
agent_builder.add_conditional_edges(
    "llm_call",
    should_continue,
    ["tool_node", END]
)
agent_builder.add_edge("tool_node", "llm_call")

# Compile the agent
agent = agent_builder.compile()

## 8. Execution

Finally, we can test our agent with a query that requires multiple steps (addition followed by multiplication).

In [ ]:
print("\n--- Running Agent (Groq) ---\n")

# Example Query
user_query = "Add 3 and 4, then multiply the result by 10."
print(f"User: {user_query}")

user_input = [HumanMessage(content=user_query)]

# Run the graph
output = agent.invoke({"messages": user_input})

print("\n--- Conversation History ---\n")
for m in output["messages"]:
    m.pretty_print()